# Lab 10.1: Tumbling Window Demo

## Tổng quan bài tập
**Đề bài**: Ở bài Lab này, bạn sẽ được hướng dẫn các thao tác để sử dụng cơ chế Tumbling Window trong Spark. Giúp tính toán tổng các giá trị Mua và Bán theo từng khoảng thời gian cách nhau **15 phút** như sau:


<img src='https://firebasestorage.googleapis.com/v0/b/funix-way.appspot.com/o/xSeries%2FData%20Engineer%2FDEP303x%2FSumary_Image%2FDEP303_sum_L14_1.png?alt=media&token=9ecf978e-6d45-4aa8-b802-5e4039fbf1d3'>


## Tài nguyên
Do bài Lab này liên quan đến đến xử lý dữ liệu Stream với Kafka, vậy nên bạn sẽ cần cài đặt Kafka, bạn có thể tham khảo video sau về cách cài đặt:
- [Cài đặt Kafka](https://funix.udemy.com/course/spark-streaming-using-python/learn/lecture/21955580#overview)

Bạn sẽ cần tải các Kafka Script ở [link sau](https://drive.google.com/file/d/1VXXedZ343pkESVlP5R8CMkX_cmqmgjJi/view?usp=sharing) để có thể sử dụng Kafka.


Bạn cũng sẽ cần tải các dữ liệu ở [link sau](https://drive.google.com/file/d/1MYqXxRfGY9MxWamvCXTh9XnsFnAIwwGz/view?usp=sharing) để có thể kiểm thử cho bài Lab.

Ngoài ra, bạn có thể tham khảo các video sau trong trường hợp chưa hiểu cách làm bài Lab:
- [Tumbling Window](https://funix.udemy.com/course/spark-streaming-using-python/learn/lecture/21955638#overview)


Import các Package cần thiết

In [ ]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import from_json, col, to_timestamp, window, expr, sum
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

Khởi tạo Spark Session

In [ ]:
spark = SparkSession \
	.builder \
	.appName("lab10 Tumbling Window Demo") \
	.master("local[3]") \
	.config("spark.streaming.stopGracefullyOnShutdown", "true") \
	.config("spark.sql.shuffle.partitions", 2) \
	.getOrCreate()

Tạo schema cho dữ liệu đầu vào

In [ ]:
stock_schema = StructType([
  StructField("CreatedTime", StringType()),
  StructField("Type", StringType()),
  StructField("Amount", IntegerType()),
  StructField("BrokerCode", StringType())
])

Hãy hoàn thiện các phần `[...]` để hoàn thiện đoạn code và giải quyết bài toán theo yêu cầu.

In [ ]:
# Đọc dữ liệu từ Kafka
kafka_df = spark.readStream \
	.format("kafka") \
	.option("kafka.bootstrap.servers", "localhost:9092") \
	.option("subscribe", "trades") \
	.option("startingOffsets", "earliest") \
	.load()

# Chuyển dữ liệu từ dạng JSON về MapType()
value_df = kafka_df.select(from_json(col("value").cast("string"), stock_schema).alias("value"))

# Trích xuất các dữ liệu
trade_df = value_df.select("value.*") \
	.withColumn("CreatedTime", to_timestamp(col("CreatedTime"), "yyyy-MM-dd HH:mm:ss")) \
	.withColumn("Buy", expr("case when Type == 'BUY' then Amount else 0 end")) \
	.withColumn("Sell", expr("case when Type == 'SELL' then Amount else 0 end"))

# Sử dụng cơ chế Tumbling Window
window_agg_df = trade_df \
	.groupBy(
		window(col("CreatedTime"), "15 minute")) \
	.agg(sum("Buy").alias("TotalBuy"),
			sum("Sell").alias("TotalSell"))

# Ghi dữ liệu đã xử lý
output_df = window_agg_df.select("window.start", "window.end", "TotalBuy", "TotalSell")

window_query = output_df.writeStream \
	.format("console") \
	.outputMode("update") \
	.option("checkpointLocation", "chk-point-dir") \
	.trigger(processingTime="1 minute") \
	.start()

print("Waiting for Query")
window_query.awaitTermination()